In [1]:
import os
import sys
sys.path.append(os.path.abspath('') + '/..')

In [2]:
import pandas as pd 
import re
import json

from data_utils import getDF
from dataset_evaluation_utils import * 

from datetime import datetime, timezone

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

# import plotly.offline as py
# pd.options.plotting.backend = "plotly"
# py.init_notebook_mode() # graphs charts inline (IPython).

a4_dims = (11.7, 8.27)

# paths

In [3]:
dataset_folderpath = '../datasets/30Music/'

dataset_name = '30Music'
dump_foldername ='30music_dump/'

# rule: what/which_data_set/sample_version/what/
images_path, output_path, heatmaps_path, diversity_graphpath, diversity_filepath = get_folderpaths(dump_foldername)

https://notebook.community/chengsoonong/digbeta/dchen/music/30music_idomaar2csv


use json.loads(str_dict) to read the str of a dictionary

In [4]:
columns = ['Type', 'ID', 'Timestamp', 'Properties', 'LinkedEntities']

# DF albums

In [4]:
albums = pd.read_csv(dataset_folderpath+'entities/albums.idomaar', delimiter='\t', header=None)
albums.columns = columns
albums.head()

,Type,ID,Timestamp,Properties,LinkedEntities
0,album,0,-1,"{""MBID"":""81a775f4-4a0e-401f-b167-f8dd97453193""...",{}
1,album,1,-1,"{""MBID"":""189bd32e-65ec-46d6-b939-6e4988ac6872""...",{}
2,album,2,-1,"{""MBID"":""8cc6207e-cfc2-40d9-a564-0d64aed50b0d""...",{}
3,album,3,-1,"{""MBID"":""38d1a53d-ed44-4924-b1b8-bd1504e61e6c""...",{}
4,album,4,-1,"{""MBID"":""442444dc-b87a-4084-b049-e6015b1ba343""...",{}


# DF events

In [12]:
# colunms = ['Type', 'ID', 'Timestamp', 'Properties', 'LinkedEntities']
events = pd.read_csv(dataset_folderpath+'relations/events.idomaar', delimiter='\t', header=None)
events.columns = columns
events.head()

,Type,ID,Timestamp,Properties,LinkedEntities
0,event.play,1,1405079768,"{""playtime"":176}","{""subjects"":[{""type"":""user"",""id"":41504}], ""obj..."
1,event.play,2,1405079944,"{""playtime"":181}","{""subjects"":[{""type"":""user"",""id"":41504}], ""obj..."
2,event.play,3,1405080125,"{""playtime"":149}","{""subjects"":[{""type"":""user"",""id"":41504}], ""obj..."
3,event.play,4,1405080273,"{""playtime"":223}","{""subjects"":[{""type"":""user"",""id"":41504}], ""obj..."
4,event.play,5,1405080497,"{""playtime"":290}","{""subjects"":[{""type"":""user"",""id"":41504}], ""obj..."


In [15]:
for le in events['LinkedEntities'][:3]:
    print(le)


{"subjects":[{"type":"user","id":41504}], "objects":[{"type":"track","id":3351207}]}
{"subjects":[{"type":"user","id":41504}], "objects":[{"type":"track","id":3351181}]}
{"subjects":[{"type":"user","id":41504}], "objects":[{"type":"track","id":3351149}]}


In [21]:
json.loads(events['LinkedEntities'][0])

{'subjects': [{'type': 'user', 'id': 41504}],
 'objects': [{'type': 'track', 'id': 3351207}]}

# DF tags

In [26]:
tags = pd.read_csv(dataset_folderpath+'entities/tags.idomaar', delimiter='\t', header=None)
tags.columns = columns
tags.head()

,Type,ID,Timestamp,Properties,LinkedEntities
0,tag,152244,-1,"{""value"":""m-dibango"", ""url"":""www.last.fm/tag/m...",{}
1,tag,58983,-1,"{""value"":""demonios da garoa"", ""url"":""www.last....",{}
2,tag,263526,-1,"{""value"":""violon-celle"", ""url"":""www.last.fm/ta...",{}
3,tag,197824,-1,"{""value"":""radio critical rachel"", ""url"":""www.l...",{}
4,tag,75705,-1,"{""value"":""eurovision 1957"", ""url"":""www.last.fm...",{}


In [27]:
for p in tags['Properties'][:3]:
    print(p)

{"value":"m-dibango", "url":"www.last.fm/tag/m-dibango"}
{"value":"demonios da garoa", "url":"www.last.fm/tag/demonios%20da%20garoa"}
{"value":"violon-celle", "url":"www.last.fm/tag/violon-celle"}


# DF sessions

In [5]:
sessions = pd.read_csv(dataset_folderpath+'relations/sessions.idomaar', delimiter='\t', header=None)
sessions.columns = ['Type', 'ID', 'Timestamp', 'P_LE']

properties = []
linked_entities = []
for row in sessions.loc[:, 'P_LE']:
    parse = row.split('} {')
    properties += [parse[0]+'}']
    linked_entities += ['{'+parse[1]]

sessions['Properties'] = properties
sessions['LinkedEntities'] = linked_entities
sessions.drop(columns=['P_LE'], inplace=True)

sessions.head()

,Type,ID,Timestamp,Properties,LinkedEntities
0,event.session,287144,1390231051,"{""numtracks"":23,""playtime"":4547}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
1,event.session,287145,1390241844,"{""numtracks"":11,""playtime"":2907}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
2,event.session,287146,1390303249,"{""numtracks"":16,""playtime"":3191}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
3,event.session,287147,1390481828,"{""numtracks"":5,""playtime"":1162}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
4,event.session,287140,1421443687,"{""numtracks"":2,""playtime"":250}","{""subjects"":[{""type"":""user"",""id"":42773}],""obje..."


In [11]:
json.loads(sessions['LinkedEntities'][0])

{'subjects': [{'type': 'user', 'id': 44361}],
 'objects': [{'type': 'track',
   'id': 4698874,
   'playstart': 0,
   'playtime': 58,
   'playratio': None,
   'action': 'play'},
  {'type': 'track',
   'id': 838286,
   'playstart': 58,
   'playtime': 310,
   'playratio': 1.01,
   'action': 'play'},
  {'type': 'track',
   'id': 2588097,
   'playstart': 368,
   'playtime': 179,
   'playratio': 1.01,
   'action': 'play'},
  {'type': 'track',
   'id': 2746740,
   'playstart': 547,
   'playtime': 342,
   'playratio': 1.44,
   'action': 'play'},
  {'type': 'track',
   'id': 3873988,
   'playstart': 889,
   'playtime': 259,
   'playratio': 1.01,
   'action': 'play'},
  {'type': 'track',
   'id': 808046,
   'playstart': 1148,
   'playtime': 236,
   'playratio': 1.01,
   'action': 'play'},
  {'type': 'track',
   'id': 2588498,
   'playstart': 1384,
   'playtime': 158,
   'playratio': 1.01,
   'action': 'play'},
  {'type': 'track',
   'id': 2492893,
   'playstart': 1542,
   'playtime': 196,
   'pl

In [9]:
json.loads(sessions['LinkedEntities'][1])

{'subjects': [{'type': 'user', 'id': 44361}],
 'objects': [{'type': 'track',
   'id': 455834,
   'playstart': 0,
   'playtime': 239,
   'playratio': 1.01,
   'action': 'play'},
  {'type': 'track',
   'id': 2460503,
   'playstart': 239,
   'playtime': 397,
   'playratio': 0.18,
   'action': 'play'},
  {'type': 'track',
   'id': 1999451,
   'playstart': 636,
   'playtime': 293,
   'playratio': 1.02,
   'action': 'play'},
  {'type': 'track',
   'id': 3257000,
   'playstart': 929,
   'playtime': 137,
   'playratio': 0.33,
   'action': 'play'},
  {'type': 'track',
   'id': 2765459,
   'playstart': 1066,
   'playtime': 186,
   'playratio': 1.06,
   'action': 'play'},
  {'type': 'track',
   'id': 4698882,
   'playstart': 1252,
   'playtime': 364,
   'playratio': None,
   'action': 'play'},
  {'type': 'track',
   'id': 1311408,
   'playstart': 1616,
   'playtime': 178,
   'playratio': 0.89,
   'action': 'play'},
  {'type': 'track',
   'id': 3439451,
   'playstart': 1794,
   'playtime': 223,
  

In [17]:
_ = json.loads(sessions['LinkedEntities'][1])
pd.json_normalize(_)

,subjects,objects
0,"[{'type': 'user', 'id': 44361}]","[{'type': 'track', 'id': 455834, 'playstart': ..."


In [7]:
_ = json.loads(sessions['LinkedEntities'][1])
pd.json_normalize(_, record_path=['objects'])#, meta=[['subjects', 'type', 'id'],['type', 'id', 'playstart', 'playtime', 'playratio', 'action']])
# pd.json_normalize(_, record_path=['subjects','objects']) # doesnt work

,type,id,playstart,playtime,playratio,action
0,track,455834,0,239,1.01,play
1,track,2460503,239,397,0.18,play
2,track,1999451,636,293,1.02,play
3,track,3257000,929,137,0.33,play
4,track,2765459,1066,186,1.06,play
5,track,4698882,1252,364,NaN,play
6,track,1311408,1616,178,0.89,play
7,track,3439451,1794,223,1.02,play
8,track,3295570,2017,106,0.39,play
9,track,2503246,2123,785,1.00,play


In [30]:
_s = pd.json_normalize(_, record_path=['subjects'])
_s

,type,id
0,user,44361


In [31]:
pd.concat([_s] * 11, axis=0)#.reset_index().drop(columns='index')

,type,id
0,user,44361
0,user,44361
0,user,44361
0,user,44361
0,user,44361
0,user,44361
0,user,44361
0,user,44361
0,user,44361
0,user,44361


In [32]:
_s = _s.loc[_s.index.repeat(11)].reset_index().drop(columns='index')#, inplace=True)
_s.columns = ['subject_type', 'subject_id']

In [34]:
_s

,subject_type,subject_id
0,user,44361
1,user,44361
2,user,44361
3,user,44361
4,user,44361
5,user,44361
6,user,44361
7,user,44361
8,user,44361
9,user,44361


In [36]:
pd.concat([ pd.json_normalize(_, record_path=['objects']), _s], axis=1)

,type,id,playstart,playtime,playratio,action,subject_type,subject_id
0,track,455834,0,239,1.01,play,user,44361
1,track,2460503,239,397,0.18,play,user,44361
2,track,1999451,636,293,1.02,play,user,44361
3,track,3257000,929,137,0.33,play,user,44361
4,track,2765459,1066,186,1.06,play,user,44361
5,track,4698882,1252,364,NaN,play,user,44361
6,track,1311408,1616,178,0.89,play,user,44361
7,track,3439451,1794,223,1.02,play,user,44361
8,track,3295570,2017,106,0.39,play,user,44361
9,track,2503246,2123,785,1.00,play,user,44361


In [ ]:
le_columns = ['subject_type', 'subjects_id', 'object_type', 'object_id', 'object_playstart', 'object_playtime', 'object_playratio', 'object_action']

# DF tracks

In [6]:
tracks = pd.read_csv(dataset_folderpath+'entities/tracks.idomaar', delimiter='\t', header=None)
tracks.columns = columns
tracks.head()

,Type,ID,Timestamp,Properties,LinkedEntities
0,track,0,-1,"{""duration"":-1,""playcount"":4,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":0}],""albums""..."
1,track,1,-1,"{""duration"":-1,""playcount"":495,""MBID"":null,""na...","{""artists"":[{""type"":""person"",""id"":1}],""albums""..."
2,track,2,-1,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":2}],""albums""..."
3,track,3,-1,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":3}],""albums""..."
4,track,4,-1,"{""duration"":-1,""playcount"":1,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":4}],""albums""..."


In [7]:
for p in tracks['Properties'][:3]:
    print(p)

for le in tracks['LinkedEntities'][:3]:
    print(le)


{"duration":-1,"playcount":4,"MBID":null,"name":"000003+Music+Instructor/_/Dj%27s+Rock+Da+House+%C3%82%E2%89%88%C3%86%E2%89%88%C3%8A01+-+Dj+Max-Pulemet+Vs.+Bomfunk+Mc%27s+-+Electro+Breakdance+party+1+%5B2000%5D+=+CD+ONE%C3%82%E2%89%88%C3%86%E2%89%88%C3%8A"}
{"duration":-1,"playcount":495,"MBID":null,"name":"00-01/_/%D0%A2%D0%B5%D0%BA%D1%81%D1%82"}
{"duration":-1,"playcount":2,"MBID":null,"name":"0005.+Overkill/_/Overkill"}
{"artists":[{"type":"person","id":0}],"albums":[],"tags":[]}
{"artists":[{"type":"person","id":1}],"albums":[],"tags":[]}
{"artists":[{"type":"person","id":2}],"albums":[],"tags":[]}
